In [1]:
import numpy as np
import torch

import dgl

import urllib.request
import pandas as pd


Using backend: pytorch


В `DGL` граф - объект класс `DGLGraph`. Для его создания нужно указать:
* кол-во вершин (узлы нумеруются, начиная с 0; кол-во вершин можно не задавать, если они все перечислены в списках для создания ребер)
* список начальных узлов для ребер
* список конечных узлов для ребер
    * узлы нумеруются, начиная с 0

`DGLGraph` всегда ориентированный. Сделать граф неориентированным можно, воспользовавшись специальным преобразованием.

На узлах (`.ndata`) и ребрах (`.edata`) могут храниться фичи:
* только числовые тензоры
* атрибуты всех узлов (ребер) должны иметь одинаковый размер

[DGLGraph API](https://docs.dgl.ai/api/python/dgl.DGLGraph.html#dgl.DGLGraph)

In [15]:
G = dgl.graph(([0, 0, 0, 0, 0], [1, 2, 3, 4, 5]), num_nodes=6)
G.ndata['x'] = torch.randn(6, 3)
G.ndata['y'] = torch.randint(0, 2, (6, ))

print('Узлы: ', G.nodes())
print('Ребра:', G.edges())
print('Фичи узлов: ', G.ndata)

print("Кол-во узлов:", G.num_nodes())
print("Кол-во ребер:", G.num_edges())
print("In-degree:", G.in_degrees())


Узлы:  tensor([0, 1, 2, 3, 4, 5])
Ребра: (tensor([0, 0, 0, 0, 0]), tensor([1, 2, 3, 4, 5]))
Фичи узлов:  {'x': tensor([[-0.4655,  0.4201, -1.2988],
        [ 0.2064,  0.9539, -0.6248],
        [-0.5796,  0.5022, -0.8503],
        [ 0.2421,  0.7596,  0.1702],
        [-0.2405,  1.5044, -0.8841],
        [ 0.9652,  1.2295, -0.0810]]), 'y': tensor([1, 1, 0, 0, 0, 0])}
Кол-во узлов: 6
Кол-во ребер: 5
In-degree: tensor([0, 1, 1, 1, 1, 1])


`DGL` предоставляет различные преобразования:
* извлечение подграфа (по узлам или по связям)
    * узлы в подграфах перенумерованы; для поиска соответствующего узла (связи) в исходном графе используем ключ `dgl.NID` (`dgl.EID`)
    * подграф сохраняет фичи исходных узлов (ребер)
* добавление обратных связей
* ...

In [25]:
SG1 = G.subgraph([0, 1, 3]) # подграф на основе узлов 0, 1 и 3
SG2 = G.edge_subgraph([0, 1, 3])  # подграф на основе ребер 0, 1 и 3

print("Соответствие узлов: ")
print(*[f'{x} -> {y}' for x, y in zip(SG1.nodes(), SG1.ndata[dgl.NID])], sep='\n')

print('Фичи узлов: ', SG1.ndata)


Соответствие узлов: 
0 -> 0
1 -> 1
2 -> 3
Фичи узлов:  {'x': tensor([[-0.4655,  0.4201, -1.2988],
        [ 0.2064,  0.9539, -0.6248],
        [ 0.2421,  0.7596,  0.1702]]), 'y': tensor([1, 1, 0]), '_ID': tensor([0, 1, 3])}


In [27]:
# делаем граф неориентированным
H = dgl.add_reverse_edges(G)
H.edges()

(tensor([0, 0, 0, 0, 0, 1, 2, 3, 4, 5]),
 tensor([1, 2, 3, 4, 5, 0, 0, 0, 0, 0]))

Для сохранения и загрузки существуют `dgl.save_graphs` и `dgl.load_graphs`

Варианты графов:
* мультиграф: более 1 ребра между одной парой узлов
* граф знаний (heterogenious graph) `dgl.heterograph`

In [29]:
G = dgl.graph((torch.tensor([0, 1, 1]), torch.tensor([1, 3, 3])))
G.is_multigraph

True

In [30]:
data_dict = {
    ('user', 'follows', 'user'): (torch.tensor([0, 1]), torch.tensor([1, 2])),
    ('user', 'follows', 'topic'): (torch.tensor([1, 1]), torch.tensor([1, 2])),
    ('user', 'plays', 'game'): (torch.tensor([0, 3]), torch.tensor([3, 4]))
}
# если не передать аргумент `num_nodes_dict`, то найдет макс. индекс узла I типа T и будет считать, что
# есть I+1 узел этого типа (даже если нет ребер)
G = dgl.heterograph(data_dict)
G


Graph(num_nodes={'game': 5, 'topic': 3, 'user': 4},
      num_edges={('user', 'follows', 'topic'): 2, ('user', 'follows', 'user'): 2, ('user', 'plays', 'game'): 2},
      metagraph=[('user', 'topic', 'follows'), ('user', 'user', 'follows'), ('user', 'game', 'plays')])

Есть возможность создавать кастомные датасеты. Для этого наследуемся от `dgl.data.DGLDataset` и реализуем три метода:
* `__getitem__`
* `__len__`
* `process` (загрузка и обработка данных с диска)

In [5]:
urllib.request.urlretrieve(
    'https://data.dgl.ai/tutorial/dataset/members.csv', './assets/data/members.csv')
urllib.request.urlretrieve(
    'https://data.dgl.ai/tutorial/dataset/interactions.csv', './assets/data/interactions.csv')

members = pd.read_csv('./assets/data/members.csv')
print(members.head())

interactions = pd.read_csv('./assets/data/interactions.csv')
print(interactions.head())


   Id    Club  Age
0   0  Mr. Hi   44
1   1  Mr. Hi   37
2   2  Mr. Hi   37
3   3  Mr. Hi   40
4   4  Mr. Hi   30
   Src  Dst    Weight
0    0    1  0.043591
1    0    2  0.282119
2    0    3  0.370293
3    0    4  0.730570
4    0    5  0.821187


In [11]:
class KarateDataset(dgl.data.DGLDataset):
    def __init__(self):
        super().__init__(name='karate_club')

    def process(self):
        # считываем файлы
        nodes_data = pd.read_csv('./assets/data/members.csv')
        edges_data = pd.read_csv('./assets/data/interactions.csv')
 
        # строим граф
        edges_src = torch.from_numpy(edges_data['Src'].to_numpy())
        edges_dst = torch.from_numpy(edges_data['Dst'].to_numpy())

        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=nodes_data.shape[0])
        # добавляем фичи
        self.graph.ndata['feat'] = torch.from_numpy(nodes_data['Age'].to_numpy())
        self.graph.ndata['label'] = torch.from_numpy(nodes_data['Club'].astype('category').cat.codes.to_numpy())
        self.graph.edata['weight'] = torch.from_numpy(edges_data['Weight'].to_numpy())

        # добавим маски для обучающего, валидационного и тестового множества
        n_nodes = nodes_data.shape[0]
        n_train = int(n_nodes * 0.6)
        n_val = int(n_nodes * 0.2)
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        val_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[:n_train] = True
        val_mask[n_train:n_train + n_val] = True
        test_mask[n_train + n_val:] = True
        self.graph.ndata['train_mask'] = train_mask
        self.graph.ndata['val_mask'] = val_mask
        self.graph.ndata['test_mask'] = test_mask

    def __getitem__(self, i):
        return self.graph

    def __len__(self):
        return 1


In [12]:
dataset = KarateDataset()
graph = dataset[0]

print(graph)


Graph(num_nodes=34, num_edges=156,
      ndata_schemes={'feat': Scheme(shape=(), dtype=torch.int64), 'label': Scheme(shape=(), dtype=torch.int8), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.float64)})


<ipython-input-11-ead4b1114757>:17: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  self.graph.ndata['label'] = torch.from_numpy(nodes_data['Club'].astype('category').cat.codes.to_numpy())


Аналогичным образом создается датасет, состоящих из нескольких графов